In [22]:
import os, time, exifread, re
from hachoir.metadata import extractMetadata
from hachoir.parser import createParser
from hachoir.core import config

In [23]:
config.quiet = True

In [30]:
def rename_file(file, c_time):
    c_time = re.sub('-|:', '', c_time).replace(' ', '_')
    file_type = file.split('.')[-1]
    new_name = f'IMG_{c_time}.{file_type}'
    if os.path.exists(new_name):
        print('File exists')
        now = str(time.time())[-5:]
        final_name = f'IMG_{c_time}_{now}.{file_type}'
    else:
        final_name = new_name
    # os.rename(file, final_name)
    print(f'{file} --> {final_name}')
    return final_name

def rename_bmtime(file, t='m'):
    if t == 'b':
        t_stamp = os.stat(file).st_birthtime
    else:
        t_stamp = os.path.getmtime(file)
    c_time = time.strftime('%Y%m%d_%H%M%S',time.localtime(t_stamp))
    new_name = rename_file(file, c_time)

def rename_img(file):
    try:
        with open(file, 'rb') as f:
            tags = exifread.process_file(f)
            c_time = str(tags['EXIF DateTimeOriginal'])
            new_name = rename_file(file, c_time)
    except KeyError:
            rename_bmtime(file, 'b')
    except:
            return file

def rename_vid(file):
    try:
        c_time = extractMetadata(createParser(file)).exportDictionary()['Metadata']['Creation date']
        if c_time[:4] != '1904':
            new_name = rename_file(file, c_time)
    except AttributeError:
            print(f'No Creation Time: {file}')
            rename_bmtime(file)
            print('-'*50)
    except:
            return file

In [43]:
files = os.listdir()

errors, others = [], []
for file in files:
    # images
    if re.search('HEIC|JPG|JPEG|PNG', file, re.IGNORECASE):
        errors.append(rename_img(file))
            
    # videos and others   
    elif re.search('MOV|MP4|M4V', file, re.IGNORECASE):
        errors.append(rename_vid(file))

    else:
        others.append(file)

errors = list(filter(None, errors))
print()
print(f'Files: {len(files)}')
print(f'Errors: {len(errors)}', errors)
print(f'Others: {len(others)}', others)

IMG_9991.HEIC --> IMG_20221002_101717.HEIC
IMG_9990.HEIC --> IMG_20221002_101712.HEIC
IMG_9986.HEIC --> IMG_20221001_172601.HEIC
IMG_9989.JPG --> IMG_20221001_172920.JPG
MTVideo_2.mp4 --> IMG_20221016_050606.mp4
MTVideo_1.mp4 --> IMG_20221022_124831.mp4
No Creation Time: IMG_9978.MOV
IMG_9978.MOV --> IMG_20221001_171012.MOV
--------------------------------------------------
IMG_9981.HEIC --> IMG_20221001_172532.HEIC
IMG_9978.HEIC --> IMG_20221001_171013.HEIC
IMG_9997.HEIC --> IMG_20221002_113849.HEIC
IMG_9986.MOV --> IMG_20221001_092601.MOV
IMG_9990.MOV --> IMG_20221002_021712.MOV
IMG_9984.MOV --> IMG_20221001_092548.MOV
IMG_9985.MOV --> IMG_20221001_092548.MOV
IMG_9991.MOV --> IMG_20221002_024221.MOV
IMG_9995.MOV --> IMG_20221002_024101.MOV
IMG_9981.MOV --> IMG_20221001_092534.MOV
IMG_9980.MOV --> IMG_20221001_092506.MOV
IMG_9994.MOV --> IMG_20221002_024130.MOV
IMG_9982.MOV --> IMG_20221001_092535.MOV
MTVideo.mp4 --> IMG_20221013_115300.mp4
sd1665731666_2.mp4 --> IMG_20221014_071420.m

# saved wechat files

In [ ]:
# rename from time_stamp in file name saved from wechat 
for i in files:
    try:
        start = re.search('16',i).start()
        c_time = time.strftime('%Y%m%d_%H%M%S', time.localtime(int(i[start:start+10])))
        new_name = rename_file(i, c_time)
        print(i, new_name)
        # os.rename(i, new_name)
    except:
        print(i)